***
# 2. Exploring Search Algorithms

This notebook details my explorations of algorithms that can solve the permutation puzzle.
***


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santa-2023/sample_submission.csv
/kaggle/input/santa-2023/puzzles.csv
/kaggle/input/santa-2023/puzzle_info.csv


In [2]:
import ast
import random

# Define a class for puzzles and a class method to create and access an allowed moves table

class AllowedMoves:
    moves_table = {}
    
    # Load the allowed moves for a puzzle type into a table
    @classmethod
    def load_moves_table(cls, puzzle_info):
        
        # Convert string representations to actual dictionaries
        puzzle_info['allowed_moves'] = puzzle_info['allowed_moves'].apply(ast.literal_eval)
        allowed_moves_dict = puzzle_info['allowed_moves'].to_dict()

        # Create all the inverse moves and load into the table
        inverse_moves_dict = {}
        for puzzle_type, puzzle_moves in allowed_moves_dict.items():
            inverse_moves = {}
            for move_name, move_array in puzzle_moves.items():
                inverse_moves[f"-{move_name}"] = np.argsort(move_array).tolist()
            inverse_moves_dict[puzzle_type] = inverse_moves

        # Create moves_table
        cls.moves_table = {puzzle_type: {**allowed_moves_dict[puzzle_type], **inverse_moves_dict[puzzle_type]}
            for puzzle_type in allowed_moves_dict}
        
    
    # Return the allowed moves for a particular puzzle type from the moves table
    @classmethod
    def get_moves(cls, puzzle_type):
        return cls.moves_table[puzzle_type]
    

class Puzzle:
    # Initialize an instance of a puzzle type and its allowed moves
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.id = self.puzzle.name
        self.type = self.puzzle['puzzle_type']
        self.initial_state = np.array(self.puzzle['initial_state'].split(';'))
        self.solution_state = np.array(self.puzzle['solution_state'].split(';'))
        self.puzzle_moves = AllowedMoves.get_moves(self.type)
        self.allowed_moves = list(self.puzzle_moves.keys())
    
    # Apply a specified move to the current state of the puzzle
    def move(self, state, move):
        if move in self.puzzle_moves:
            result_state = state[np.array(self.puzzle_moves[move])]
        else:
            print(f"Invalid move: {move}")
            result_state = state
        
        return result_state


***
## Ideas about algorithms

I began thinking about algorithms with a visit to Robert Sedgewick and Kevin Wayne's book "Algorithms". I confirmed my thoughts that a graph search algorithm would serve me well for this type of permutation puzzle. The initial and solution states would be vertices of the graph and the allowed moves would be the edges of the graph. Given an initial vertex and a solution vertex, I will need to find the shortest path between them, that is, the path with the fewest number of edges, or moves.  
  
I decided to start off by employing a basic Breadth First Search method. This is a 'brute-force' method that considers every possible permutation of moves until a valid solution is achieved.
***

In [3]:
from queue import Queue
from time import time

# Load data
puzzle_info = pd.read_csv('/kaggle/input/santa-2023/puzzle_info.csv', index_col='puzzle_type')
puzzles = pd.read_csv('/kaggle/input/santa-2023/puzzles.csv', index_col='id')
sample_submission = pd.read_csv('/kaggle/input/santa-2023/sample_submission.csv', index_col='id')
final_submission = pd.DataFrame(columns=['id', 'moves'])

# Load puzzle_info into the allowed moves table
AllowedMoves.load_moves_table(puzzle_info)


# Main program control logic to solve each puzzle in turn - initially, I am only tackling the first two puzzles

# for i in range(len(puzzles)):   - This is the code that runs on ALL puzzles to use later
# The following code just runs on the first 2 puzzles

for i in range(2):
    # Start a timer for solving the puzzle
    start_time = time()
    
    # Create puzzle object
    puzzle = Puzzle(puzzles.iloc[i])  
    
    # Initialize the queue and tracking variables
    q = Queue()
    q.put((puzzle.initial_state, []))
    visited = set()
    random.shuffle(puzzle.allowed_moves)
    depth = 3  # Using this constant to vary the maximum search depth of the algorithm
    solution_found = False
    
    while not q.empty():
        current_state, current_path = q.get()
        if len(current_path) > depth:
            break
                
        if np.array_equal(current_state, puzzle.solution_state):
            solution_found = True
            solution_path = '.'.join(current_path)
                    
            # Record the solution path for the puzzle
            puzzle_solution = pd.DataFrame([{'id': puzzle.id, 'moves': solution_path}])
            final_submission = pd.concat([final_submission, puzzle_solution])    
            break

        if tuple(current_state) in visited:
            continue

        visited.add(tuple(current_state))
                
        for move in puzzle.allowed_moves:
            new_state = puzzle.move(current_state, move)
            new_path = current_path + [move]
            q.put((new_state, new_path))
                    
        random.shuffle(puzzle.allowed_moves)
           
    if not solution_found:
        print(f"Puzzle id_{puzzle.id}: Solution not found")
        print(f"Incomplete path: {current_path}\n")
    else:
        # Stop the timer and print the time taken to solve the puzzle
        end_time = time()
        elapsed_time = end_time - start_time
        print(f"Puzzle id_{puzzle.id}: Solution found in {elapsed_time} seconds")
        print(f"Solution path: {current_path}\n")

# Check contents of the DataFrame the records the puzzle solutions found
print("Final submission:")
print(final_submission.set_index('id'))

Puzzle id_0: Solution found in 0.021597862243652344 seconds
Solution path: ['r1', '-f1']

Puzzle id_1: Solution not found
Incomplete path: ['d1', 'f0', '-d1', 'd1']

Final submission:
     moves
id        
0   r1.-f1


***
The first puzzle turns out to be simple. The BFS algorithm finds the solution in 2 moves with an elapsed time of under a second. A little research into Rubrick's cube puzzles reveals that the God's number for a 2x2 cube is 11 - this number provides a worst-case upper bound on the shortest path length to the solution state. It quantifies the maximum complexity from any initial state. The second puzzle remained unsolved after trying all permutations of 8 consecutive moves in around 11 minutes.

The elapsed time to solve the 2x2 cube puzzle increases exponentially with every additional required move as the search space increases: 12, 12^2, 12^3 etc. The brute-force method of Breadth First Search is impractical for any cube puzzle bar the simplest/smallest.

As an alternative, I am going to try implementing the A* search algorithm. 
***